<a href="https://colab.research.google.com/github/JuliaPanov/For/blob/main/Foraminifera_RandomForest_Jan2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Random Forest Regression **

Input tables:

Y matrix is index "J" across all samples;

X matrix is encoded ecological data across all samples

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd

dfY= pd.read_csv('/content/Indexes_abundance.txt', sep='\t')
dfX=pd.read_csv('/content/Ecological_CategoricalData_Filt_T_Morphotype_Encoded.txt', sep='\t')


In [ ]:
# Matrix Y
Y = dfY['J']
#print(Y)
#Y.shape

In [ ]:
X = dfX.drop('ID', axis=1)
#X.shape

In [76]:
feature_list = list(X.columns)
#print(feature_list)

['Site_AK', 'Site_RD1', 'Site_SP1', 'Depth_(m)_5', 'Depth_(m)_8', 'Depth_(m)_18', 'Season_Fall', 'Season_Spring', 'Season_Summer', 'Season_Winter', 'Morphotype_articulated_calcareous', 'Morphotype_corticated', 'Morphotype_corticated_foliose', 'Morphotype_leathery']


In [36]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [37]:
##RANDOM FOREST REGRESSION --> index "J"
RF = RandomForestRegressor(max_depth=3, n_estimators = 100, bootstrap=True)
RF.fit(X_test, y_test)
RF_predict = RF.predict(X_test)

In [54]:
# save the model in a binary file to be used later if needed
filename = 'RF_model.sav'
with open(filename, 'wb') as f:
    pickle.dump(RF, f)

In [ ]:
# Plot the results of predictions
import matplotlib.pyplot as plt
x_ax = range(len(y_test))
plt.figure()
plt.scatter(x_ax, y_test, s=100, c="cornflowerblue", label="data")
plt.scatter(x_ax, RF_predict, s=20, edgecolor="black", color="darkorange", label="prediction", linewidth=2)
plt.xlabel("Observation")
plt.ylabel("J index")
plt.title("Random Forest Regression")
plt.legend()

# Save the plot as a PNG file
plt.savefig("random_forest_regression_plot.png")

# Show the plot
#plt.show()

In [50]:
with open('RandomForest_Accuracy.txt', 'w') as file:
    file.write("Model Accuracy: %.3f\n" % RF.score(X_test, y_test))
    mse = mean_squared_error(y_test, RF.predict(X_test))
    file.write("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
##get feature importance
feature_importance = RF.feature_importances_

importance_df = pd.DataFrame({'features': X_train.columns,
                              'importance': feature_importance})
importance_df.sort_values(by='importance', ascending=False, inplace=True)
importance_df
importance_df.to_csv('RandomForestRegressor_FeatureImportance.txt', index=True, sep='\t')
#print(importance_df)

In [57]:
# Trees - visualization
from sklearn.tree import export_graphviz
from IPython.display import Image, display
import graphviz
import os

# Create a directory to save the trees
if not os.path.exists("saved_trees"):
    os.makedirs("saved_trees")

# Export the first i decision trees from the forest
for i in range(5):
    tree = RF.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,
                               filled=True,
                               max_depth=2,
                               impurity=False,
                               proportion=True)

    # Save the plot as a PNG file with a unique filename for each tree
    filename = f"saved_tree_{i+1}.png"
    graph = graphviz.Source(dot_data, format="png")
    graph.render(filename, format="png", cleanup=True)

    # Display the graph if needed
    #display(graph)


In [ ]:
# For later use --> load the model
filename = 'RF_Regression_model.sav'
with open(filename, 'rb') as f:
    #pickle.dump(model, f)
    loaded_model = pickle.load(f)
result = loaded_model.score(X_test, Y_test)
#print(result)



*Random Forest Classifier *

Input tables:

Y matrix is index "Class" based on the depth of the observation;

X matrix is relative abundance of foraminifera dominant specaies that appear in more than 5 observations



In [ ]:
df=pd.read_csv('/Depth_class_abundance_Forams5samples.txt', sep='\t')

# Matrix Y
Y = df['Class']
#Y.shape

In [ ]:
# Matrix X

X = df.drop('Class', axis=1)
#X.shape

In [71]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
##RANDOM FOREST CLASSIFIER
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5, criterion ='gini', min_samples_leaf= 3, bootstrap=True)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Make predictions on the test set
predictions = rf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy_train = accuracy_score(y_train, rf.predict(X_train))

with open('RandomForest_Accuracy.txt', 'w') as file:
    file.write((f"Accuracy: {accuracy:.2f}"))
    file.write(f"Accuracy on training set: {accuracy_train:.2f}")

print(f"Accuracy on testing set: {accuracy:.2f}")
print(f"Accuracy on training set: {accuracy_train:.2f}")


In [91]:
# save the model in a binary file to be used later if needed
filename = 'RF_model.sav'
with open(filename, 'wb') as f:
    pickle.dump(RF, f)

In [ ]:
# Trees - visualization
from sklearn.tree import export_graphviz
from IPython.display import Image, display
import graphviz
import os

# Create a directory to save the trees
if not os.path.exists("saved_trees"):
    os.makedirs("saved_trees")

# Export the first i decision trees from the forest
for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_test.columns,
                               filled=True,
                               max_depth=2,
                               impurity=False,
                               proportion=True)

    # Save the plot as a PNG file with a unique filename for each tree
    filename = f"saved_tree_{i+1}.png"
    graph = graphviz.Source(dot_data, format="png")
    graph.render(filename, format="png", cleanup=True)

    # Display the graph if needed
    #display(graph)

In [ ]:
##get feature importance
feature_importance = rf.feature_importances_

importance_df = pd.DataFrame({'features': X_train.columns,
                              'importance': feature_importance})
importance_df.sort_values(by='importance', ascending=False, inplace=True)
importance_df
importance_df.to_csv('RandomForest_FeatureImportance.txt', index=True, sep='\t')
#print(importance_df)

In [ ]:
##CONFUSION MATRIX
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['I', 'SH', 'VS'], yticklabels=['I', 'SH', 'VS'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')

# Save the figure
plt.savefig('confusion_matrix.png')

# Show the plot
#plt.show()